In [1]:
import san

san.available_metrics_for_slug("bitcoin")

['30d_moving_avg_dev_activity_change_1d',
 'active_addresses_1h',
 'active_addresses_24h',
 'active_addresses_24h_change_1d',
 'active_addresses_24h_change_30d',
 'active_addresses_24h_change_7d',
 'active_addresses_30d',
 'active_addresses_7d',
 'active_holders_distribution_0.001_to_0.01',
 'active_holders_distribution_0.01_to_0.1',
 'active_holders_distribution_0.1_to_1',
 'active_holders_distribution_0_to_0.001',
 'active_holders_distribution_100_to_1k',
 'active_holders_distribution_100k_to_1M',
 'active_holders_distribution_10_to_100',
 'active_holders_distribution_10k_to_100k',
 'active_holders_distribution_1_to_10',
 'active_holders_distribution_1k_to_10k',
 'active_holders_distribution_combined_balance_0.001_to_0.01',
 'active_holders_distribution_combined_balance_0.01_to_0.1',
 'active_holders_distribution_combined_balance_0.1_to_1',
 'active_holders_distribution_combined_balance_0_to_0.001',
 'active_holders_distribution_combined_balance_100_to_1k',
 'active_holders_distribut

In [2]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="./.env")

True

In [3]:
import os
import san

santiment_api_key = os.getenv("SANTIMENT_API_KEY")
san.ApiConfig.api_key = santiment_api_key

In [4]:
san.get(
  "price_usd",
  slug="bitcoin",
  from_date="2024-04-15",
  to_date="2024-04-16",
  interval="5m"
)

,value
datetime,
2024-04-15 00:00:00+00:00,65553.991257
2024-04-15 00:05:00+00:00,65529.448231
2024-04-15 00:10:00+00:00,65890.195427
2024-04-15 00:15:00+00:00,65761.781251
2024-04-15 00:20:00+00:00,65734.916069
...,...
2024-04-16 23:35:00+00:00,63816.373483
2024-04-16 23:40:00+00:00,63797.449047
2024-04-16 23:45:00+00:00,63755.512583


In [5]:
from_date = '2024-01-01'
to_date = '2024-04-17'
asset = 'bitcoin'
interval = '1d'

# create batch object
batch = san.Batch()

# create a request
batch.get(
    f'daily_active_addresses/{asset}',
    from_date=from_date,
    to_date=to_date,
    interval=interval
)
batch.get(
    f'prices/{asset}',
    from_date=from_date,
    to_date=to_date,
    interval=interval
)

# execute the request
[daa, price] = batch.execute()

# merge dataframes
data = daa.rename(columns={'value':'activeAddresses'}).join(price['priceUsd'])

# take a look
data.head()

,activeAddresses,priceUsd
datetime,,
2024-01-01 00:00:00+00:00,831736.0,44167.330214
2024-01-02 00:00:00+00:00,873732.0,44951.138005
2024-01-03 00:00:00+00:00,954003.0,42848.175641
2024-01-04 00:00:00+00:00,1016355.0,44200.715873
2024-01-05 00:00:00+00:00,865666.0,44160.760983


In [9]:
import datetime
import pandas as pd
import numpy as np
import san
import matplotlib.pyplot as plt


def fancy_plot(df, left_metrics: list, right_metric=None, signals=[]):
    """Quick visualization tool."""
    fig, ax1 = plt.subplots(figsize=(18, 9))
    for metric in left_metrics:
        (p1,) = ax1.plot(df.index, df[metric], label=metric, linewidth=2)
        plt.grid("x")
        plt.legend(loc="upper right", fontsize=14)

    ax2 = ax1.twinx()
    if right_metric:
        (p1,) = ax2.plot(
            df.index, df[right_metric], color="red", label=right_metric, linewidth=1.8
        )
        plt.legend(loc="upper left", fontsize=14)

    plt.vlines(signals, df.min().min(), df.max().max(), color="#424242", linewidth=1)


def get_san_metric(
    start, end, metric, asset, interval, iterate_over_days=120, convert_index=True
):
    """Iterates over days and gets the metric data."""
    start = datetime.datetime.strptime(start, "%Y-%m-%d").date()
    end = datetime.datetime.strptime(end, "%Y-%m-%d").date()
    df = pd.DataFrame(None)

    for i in range(int(np.ceil((end - start).days / iterate_over_days))):
        start_date = str(start + datetime.timedelta(days=iterate_over_days * i))
        end_date = str(
            min(end, start + datetime.timedelta(days=iterate_over_days * (i + 1) - 1))
        )

        df_batch = san.get(
            f"{metric}/{asset}",
            from_date=start_date,
            to_date=end_date,
            interval=interval,
        )
        df = pd.concat([df, df_batch])
        if convert_index:
            df.index = df.index.tz_localize(None)

    return df

In [10]:
from_date = '2024-04-01' # skip bullrun
to_date = '2024-04-16'
asset = 'bitcoin'

ex_flow = get_san_metric(from_date, to_date, 'exchange_funds_flow', asset, interval='1d', iterate_over_days=700)
price = get_san_metric(from_date, to_date, 'prices', asset, interval='1h', iterate_over_days=120)

data = price.join(ex_flow).fillna(0)
data.head()

,marketcap,priceBtc,priceUsd,volume,inOutDifference
datetime,,,,,
2024-04-01 00:00:00,1.398215e+12,1.0,71084.711453,2.038583e+10,-9485.761101
2024-04-01 01:00:00,1.393873e+12,1.0,70863.541580,2.039884e+10,0.000000
2024-04-01 02:00:00,1.394310e+12,1.0,70885.787908,2.052555e+10,0.000000
2024-04-01 03:00:00,1.389309e+12,1.0,70631.515951,2.070726e+10,0.000000
2024-04-01 04:00:00,1.386377e+12,1.0,70482.053537,2.087550e+10,0.000000


In [11]:
san.api_calls_remaining()

{'month_remaining': '79975',
 'hour_remaining': '3986',
 'minute_remaining': '99'}

In [12]:
san.get(
    "ohlcv/bitcoin",
    from_date="2018-06-01",
    to_date="2024-04-16",
    interval="1d"
)

,openPriceUsd,closePriceUsd,highPriceUsd,lowPriceUsd,volume,marketcap
datetime,,,,,,
2018-06-01 00:00:00+00:00,7500.700000,7541.450000,7604.730000,7407.340000,4.921460e+09,1.287259e+11
2018-06-02 00:00:00+00:00,7536.720000,7643.450000,7695.830000,7497.260000,4.939300e+09,1.304815e+11
2018-06-03 00:00:00+00:00,7632.090000,7720.250000,7754.890000,7613.040000,4.851760e+09,1.318080e+11
2018-06-04 00:00:00+00:00,7722.530000,7514.470000,7753.820000,7474.040000,4.993170e+09,1.283122e+11
2018-06-05 00:00:00+00:00,7500.900000,7633.760000,7643.230000,7397.000000,4.961740e+09,1.303659e+11
...,...,...,...,...,...,...
2024-04-12 00:00:00+00:00,70032.843455,67188.376427,71195.927032,66052.556414,4.412930e+10,1.322319e+12
2024-04-13 00:00:00+00:00,67147.720119,63836.231837,67888.074910,61593.267402,5.286974e+10,1.256414e+12
2024-04-14 00:00:00+00:00,64023.673008,65738.723887,65812.108100,62214.793360,4.908432e+10,1.293922e+12


In [ ]:
san.get(
    "historical_balance",
    slug="santiment",
    address="0x1f3df0b8390bb8e9e322972c5e75583e87608ec2",
    from_date="2019-04-18",
    to_date="2019-04-23",
    interval="1d"
)